In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv


In [2]:
data="/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv"
data = pd.read_csv(data)
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [3]:
data = data.drop("ph",axis=1)

In [4]:
X = data.drop("label",axis=1)
y = data["label"]

In [5]:
y

0         rice
1         rice
2         rice
3         rice
4         rice
         ...  
2195    coffee
2196    coffee
2197    coffee
2198    coffee
2199    coffee
Name: label, Length: 2200, dtype: object

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
le = LabelEncoder()
y = le.fit_transform(y)
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [9]:
st = StandardScaler()
X = st.fit_transform(X)
X_train = st.fit_transform(X_train)
X_test = st.fit_transform(X_test)

In [10]:
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier,RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
import xgboost as xgb
modeldcr = DecisionTreeRegressor()
modelrfr = RandomForestRegressor()
modeldcr.fit(X_train,y_train)


DecisionTreeRegressor()

In [11]:
# ===============================
# Define models
# ===============================
models = [
    ("Logistic Regression", LogisticRegression(max_iter=1000)),
    ("Decision Tree", DecisionTreeClassifier(random_state=42)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("KNN", KNeighborsClassifier(n_neighbors=5)),
    ("XGBoost", xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ("Bagging", BaggingClassifier(random_state=42)),
    ("Ridge Classifier", RidgeClassifier(random_state=42)),
    ("SGD Classifier", SGDClassifier(max_iter=1000, tol=1e-3, random_state=42))
]

# ===============================
# Evaluate each model
# ===============================
Results = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1_Score'])

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    pre = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\n📊 Model: {name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {pre:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("--------------------------------------------------")

    new_row = pd.DataFrame({
        'Model': [name],
        'Accuracy': [acc],
        'Precision': [pre],
        'Recall': [rec],
        'F1_Score': [f1]
    })
    Results = pd.concat([Results, new_row], ignore_index=True)

# ===============================
# Show all results sorted by F1 score
# ===============================
Results = Results.sort_values(by="F1_Score", ascending=False)
print("\n🏆 Final Model Performance Comparison:")
print(Results)

# Optional: Display confusion matrix for best model
best_model_name = Results.iloc[0]['Model']
best_model = dict(models)[best_model_name]
best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)
print(f"\nConfusion Matrix for {best_model_name}:")
print(confusion_matrix(y_test, y_pred_best))


📊 Model: Logistic Regression
Accuracy: 0.9500
Precision: 0.9530
Recall: 0.9500
F1 Score: 0.9498
--------------------------------------------------

📊 Model: Decision Tree
Accuracy: 0.9523
Precision: 0.9621
Recall: 0.9523
F1 Score: 0.9526
--------------------------------------------------


/tmp/ipykernel_37/525327082.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Results = pd.concat([Results, new_row], ignore_index=True)



📊 Model: Random Forest
Accuracy: 0.9727
Precision: 0.9779
Recall: 0.9727
F1 Score: 0.9735
--------------------------------------------------

📊 Model: KNN
Accuracy: 0.9545
Precision: 0.9618
Recall: 0.9545
F1 Score: 0.9549
--------------------------------------------------

📊 Model: XGBoost
Accuracy: 0.9295
Precision: 0.9345
Recall: 0.9295
F1 Score: 0.9288
--------------------------------------------------

📊 Model: Bagging
Accuracy: 0.9591
Precision: 0.9695
Recall: 0.9591
F1 Score: 0.9597
--------------------------------------------------

📊 Model: Ridge Classifier
Accuracy: 0.6182
Precision: 0.6214
Recall: 0.6182
F1 Score: 0.5505
--------------------------------------------------

📊 Model: SGD Classifier
Accuracy: 0.8841
Precision: 0.9175
Recall: 0.8841
F1 Score: 0.8884
--------------------------------------------------

🏆 Final Model Performance Comparison:
                 Model  Accuracy  Precision    Recall  F1_Score
2        Random Forest  0.972727   0.977937  0.972727  0.973538

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Confusion Matrix for Random Forest:
[[23  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0 22  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0 20  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2  0  0 22  0  0  0  0  0  0  0  

In [12]:
import pickle
with open('recommend.pkl','wb') as f: 
    pickle.dump(best_model_name,f)

In [14]:
with open('recommend.pkl','rb') as f: 
    b = pickle.load(f)
    print(b)

Random Forest
